In [ ]:
# import necessary packages
from gmodels.markov import MarkovNetwork
from gmodels.gtypes.edge import Edge, EdgeType
from gmodels.factor import Factor
from gmodels.randomvariable import NumCatRVariable


In [ ]:
# define data and random variable nodes
idata = {
    "A": {"outcome-values": [True, False]},
    "B": {"outcome-values": [True, False]},
    "C": {"outcome-values": [True, False]},
    "D": {"outcome-values": [True, False]},
}
                                                                   
# misconception example: Koller, Friedman, 2009 p. 104
 
A = NumCatRVariable(
    node_id="A", input_data=idata["A"], marginal_distribution=lambda x: 0.5
)
B = NumCatRVariable(
    node_id="B", input_data=idata["B"], marginal_distribution=lambda x: 0.5
)
C = NumCatRVariable(
    node_id="C", input_data=idata["C"], marginal_distribution=lambda x: 0.5
)
D = NumCatRVariable(
    node_id="D", input_data=idata["D"], marginal_distribution=lambda x: 0.5
)

In [ ]:
# define edges
AB = Edge(
    edge_id="AB",
    edge_type=EdgeType.UNDIRECTED,
    start_node=A,
    end_node=B,
)
AD = Edge(
    edge_id="AD",
    edge_type=EdgeType.UNDIRECTED,
    start_node=A,
    end_node=D,
)
DC = Edge(
    edge_id="DC",
    edge_type=EdgeType.UNDIRECTED,
    start_node=D,
    end_node=C,
)
BC = Edge(
    edge_id="BC",
    edge_type=EdgeType.UNDIRECTED,
    start_node=B,
    end_node=C,
)

In [ ]:
# define factor functions

def phi_AB(scope_product):
    ""
    ss = frozenset(scope_product)
    if ss == frozenset([("A", False), ("B", False)]):
        return 30.0
    elif ss == frozenset([("A", False), ("B", True)]):
        return 5.0
    elif ss == frozenset([("A", True), ("B", False)]):
        return 1.0
    elif ss == frozenset([("A", True), ("B", True)]):
        return 10.0
    else:
        raise ValueError("product error")
                                                                   
def phi_BC(scope_product):
    ""
    ss = frozenset(scope_product)
    if ss == frozenset([("B", False), ("C", False)]):
        return 100.0
    elif ss == frozenset([("B", False), ("C", True)]):
        return 1.0
    elif ss == frozenset([("B", True), ("C", False)]):
        return 1.0
    elif ss == frozenset([("B", True), ("C", True)]):
        return 100.0
    else:
        raise ValueError("product error")
                                                                   
def phi_CD(scope_product):
    ""
    ss = frozenset(scope_product)
    if ss == frozenset([("C", False), ("D", False)]):
        return 1.0
    elif ss == frozenset([("C", False), ("D", True)]):
        return 100.0
    elif ss == frozenset([("C", True), ("D", False)]):
        return 100.0
    elif ss == frozenset([("C", True), ("D", True)]):
        return 1.0
    else:
        raise ValueError("product error")
                                                                   
def phi_DA(scope_product):
    ""
    ss = frozenset(scope_product)
    if ss == frozenset([("D", False), ("A", False)]):
        return 100.0
    elif ss == frozenset([("D", False), ("A", True)]):
        return 1.0
    elif ss == frozenset([("D", True), ("A", False)]):
        return 1.0
    elif ss == frozenset([("D", True), ("A", True)]):
        return 100.0
    else:
        raise ValueError("product error")


In [ ]:
# instantiate factors with factor functions and implied random variables in scope 
AB_f = Factor(
    gid="ab_f", scope_vars=set([A, B]), factor_fn=phi_AB
)
BC_f = Factor(
    gid="bc_f", scope_vars=set([B, C]), factor_fn=phi_BC
)
CD_f = Factor(
    gid="cd_f", scope_vars=set([C, D]), factor_fn=phi_CD
)
DA_f = Factor(
    gid="da_f", scope_vars=set([D, A]), factor_fn=phi_DA
)

In [ ]:
# instantiate markov network and make a query
mnetwork = MarkovNetwork(
    gid="mnet",
    nodes=set([A, B, C, D]),
    edges=set([AB, AD, BC, DC]),
    factors=set([DA_f, CD_f, BC_f, AB_f]),
)
 
queries = set([A, B])
evidences = set()
prob, a = mnetwork.cond_prod_by_variable_elimination(queries, evidences)
q2 = set([("A", False), ("B", True)])
round(prob.phi_normal(q2), 2)
# 0.69
